In [1]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores.utils import filter_complex_metadata
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Instantiate a WebBaseLoader object with a URL to a web page
loader= WebBaseLoader("https://en.wikipedia.org/wiki/Hewlett-Packard")
web_data= loader.load()

In [5]:
# Instantiate a Docx2txtLoader object with the path to a .docx file
loader= Docx2txtLoader(r"C:\Users\sdakshit\Documents\RAG\Llama\Data\FMove_Draft_07212024_Edits.docx")

doc_data1= loader.load()

In [2]:
# Define the path to a PDF file
pdf_path= (r"C:\Users\sdakshit\Documents\RAG\Llama\Data\merged.pdf")
pdf_data= PyPDFLoader(file_path = pdf_path).load()

In [3]:
folder_path= r"C:\Users\PolyStudio-ssat\PH2\WhitePaper"
# Create a DirectoryLoader instance to load .docx files from the specified folder.
# The 'glob' parameter specifies the pattern to match .docx files.
# The 'loader_cls' parameter specifies the class to use for loading the documents,
# in this case, UnstructuredWordDocumentLoader for unstructured Word documents.
txt_loader = DirectoryLoader(folder_path, glob= "*.docx", loader_cls= Docx2txtLoader)
# Load the documents using the DirectoryLoader instance and store them in txt_documents.
# This will return a list of document objects as defined by UnstructuredWordDocumentLoader.
txt_documents = txt_loader.load()

FileNotFoundError: Directory not found: 'C:\Users\PolyStudio-ssat\PH2\WhitePaper'

In [3]:
#Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# Change the data source as per file type. 
all_splits = text_splitter.split_documents(pdf_data)
chunks= filter_complex_metadata(all_splits)

In [4]:
print(len(chunks))

95


In [5]:
# Initialize a SentenceTransformerEmbeddings object with the specified model
embeddings= SentenceTransformerEmbeddings(model_name= "all-MiniLM-L6-v2")

# Create a vector database store from the document chunks
vectorstore= Chroma.from_documents(
    documents=chunks, 
    collection_name="rag-chroma",  # Name of the collection in the vector store
    embedding=embeddings  # The embeddings object to use for generating vector representations

)

C:\Users\sdakshit\AppData\Local\anaconda\envs\LLMs\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\sdakshit\AppData\Local\anaconda\envs\LLMs\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
# Initialize a retriever from the vector store with specific search parameters

retriever = vectorstore.as_retriever(
    search_type= "similarity_score_threshold",
    search_kwargs={
        "k" :3,  #Return top 3 results
        "score_threshold": 0.5    #Return results with similarity score greater than 0.5
    },
)

In [7]:
# Define a template for generating chat prompts with context and question placeholders

qa_system_prompt = """You are an assistant for HP, for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Start your answers either with 'woof woof' or 'Don't worry human, wolfy will help you'.\

{context}"""

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""


# Create a ChatPromptTemplate object from the defined template
contextualize_q_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Specify the model to use, in this case, 'llama3'
ollama_llm= "phi3"

model_local = ChatOllama(model=ollama_llm)


question_answer_chain = create_stuff_documents_chain(model_local, qa_prompt)

history_aware_retriever= create_history_aware_retriever(
    model_local, retriever, contextualize_q_prompt
)

rag_chain= create_retrieval_chain(history_aware_retriever, question_answer_chain)



store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [9]:
conversational_rag_chain.invoke({"input": "WHat is secure boot?"},
                                config={
                                    "configurable": { "session_id": "session1"}
                                    },
                                    )["answer"]

C:\Users\sdakshit\AppData\Local\anaconda\envs\LLMs\lib\site-packages\langchain_core\vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Lock Management\n●Lock and unlock requests are handled by the lock manager\n●Lock table entry:\n○Number of transactions currently holding a lock\n○Type of lock held (shared or exclusive)\n○Pointer to queue of lock requests\n●Locking and unlocking have to be atomic operations\n●Lock upgrade: transaction that holds a shared lock can be upgraded to hold an exclusive \nlock', metadata={'page': 49, 'source': 'C:\\Users\\sdakshit\\Documents\\RAG\\Llama\\Data\\merged.pdf'}), -0.02712931921631223), (Document(page_content='Deadlock Prevention', metadata={'page': 52, 'source': 'C:\\Users\\sdakshit\\Documents\\RAG\\Llama\\Data\\merged.pdf'}), -0.04536760926612837), (Document(page_content='which every write is recorded. This mechanism is also used to recover from \nsystem crashes: all active Xacts at the time of

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D704F12DD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
conversational_rag_chain.invoke({"input": "Why is it important?"},
                                config={
                                    "configurable": { "session_id": "session1"}
                                    },
                                    )["answer"]